In [6]:
from google.colab import drive 
import os
drive.mount('/content/drive')
path = "/content/drive/My Drive/Bookpred/Assignment 2"
%cd {path}
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Bookpred/Assignment 2
 8.prep_IMDB.ipynb
'Assignment 2 Track.gdoc'
 bert
 best_model
 book_rating_test.csv
 book_rating_train.csv
 book_text_features_countvec
 book_text_features_doc2vec
 book_text_features_tfidf
 COMP30027_Project2_spec_2023S1.pdf
'Copy of model.png'
 English
'English (1)'
'Feature engineering.ipynb'
 full_dataset
 history.pkl
 history_v.pkl
 lgbm_class_weight.joblib
 lgbm_normal.joblib
 lgbm_oversampling.joblib
 matrix.npz
 Model.ipynb
 model.png
 multilingual_best
 multilingual_epochs
 output.csv
 Oversampling
 Oversampling_real
 partition
 random_forest_doc.joblib
 random_forest.joblib
 random_forest_num.joblib
 small_bert
 smallbert_class_weight.png
 smallbert_focal.png
 smallbert_normal.png
 smallbert_oversampling_minority.png
 Test
 test_focal.npy
 text_features_description.ipynb
 train_ind.pkl
 Validation
 val_

# COMP30027 Machine Learning Project 2

## Description of text features

This notebook describes the pre-computed text features provided for Project 2. **You do not need to recompute the features yourself for this assignment** -- this information is just for your reference. However, feel free to experiment with different text features if you are interested. If you do want to try generating your own text features, some things to keep in mind:
- There are many different decisions you can make throughout the feature design process, from the text preprocessing to the size of the output vectors. There's no guarantee that the defaults we chose will produce the best possible text features for this classification task, so feel free to experiment with different settings.
- These features must be trained using a training corpus. Generally, the training corpus should not include validation samples, but for the purposes of this assignment we have used the entire non-test set (training+validation) as the training corpus, to allow you to experiment with different validation sets. If you recompute the text features as part of your own model, you should exclude validation samples and compute them on training samples only. For example, if you do N-fold cross-validation, this means generating N sets of features for N different training-validation splits.

In [7]:
import numpy as np
import pandas as pd

# read text
# for DEMONSTRATION PURPOSES, the entire training set will be used to train the models and also as a test set
x_train_original = pd.read_csv(r"book_rating_train.csv", index_col = False, delimiter = ',', header=0)
# use recipe name as an example
train_corpus_name = x_train_original['Name']
test_name = x_train_original['Name']

In [ ]:
os.mkdir("book_text_features_tfidf")

In [9]:
os.makedirs("partition", exist_ok = True)
os.makedirs("partition/train", exist_ok = True)
os.makedirs("partition/val", exist_ok = True)
os.makedirs("partition/test", exist_ok = True)



## Count vectorizer

A count vectorizer converts documents to vectors which represent word counts. Each column in the output represents a different word and the values indicate the number of times that word appeared in the document. The overall size of a count vector matrix can be quite large (the number of columns is the total number of different words used across all documents in a corpus), but most entries in the matrix are zero (each document contains only a few of all the possible words). Therefore, it is most efficient to represent the count vectors as a sparse matrix.

In [10]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd
import re
import nltk
from string import punctuation
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csr_matrix, save_npz, load_npz
from sklearn.preprocessing import StandardScaler, OneHotEncoder



def preprocessing_pipeline(text):
    # Remove HTML tags
  clean = re.compile('<.*?>')
  text = re.sub(clean, '', text)
  
  # Remove punctuation
  text = ''.join(c for c in text if c not in punctuation)
  
  # Tokenize text
  tokens = text.split()
  
  # Remove stopwords
  stopwords_list = stopwords.words('english')
  words = [word for word in tokens if word.lower() not in stopwords_list]
  
  # Stem words
  stemmer = PorterStemmer()
  stemmed_words = [stemmer.stem(word) for word in words]
  
  # Join cleaned and stemmed words back into sentence
  return " ".join(stemmed_words)


def create_npz_file():
  train_df = pd.read_csv(r"book_rating_train.csv", 
                         index_col = False, delimiter = ',', header=0)
  test_df = pd.read_csv(r"book_rating_test.csv", 
                        index_col = False, delimiter = ',', header=0)
  
  # features = ['Authors', 'Name', 'Description']
  features = ['Publisher']


  for feature in features:
    print(feature)
    train_target = train_df[feature].fillna("").apply(lambda x: preprocessing_pipeline(x))
    test_target = test_df[feature].fillna("").apply(lambda x: preprocessing_pipeline(x))
    # preprocess text and compute counts
    vectorizer = TfidfVectorizer(stop_words='english').fit(train_target)
        
    # generate counts for a new set of documents
    vec_train = vectorizer.transform(train_target)
    vec_test = vectorizer.transform(test_target)
    print(vec_train.shape)
    print(vec_test.shape)




    if(feature == 'Description'):
      save_npz(f'./book_text_features_tfidf/train_desc_vec.npz', vec_train)
      save_npz(f'./book_text_features_tfidf/test_desc_vec.npz', vec_test)

    else:
      save_npz(f'./book_text_features_tfidf/train_{feature.lower()}_vec.npz', vec_train)
      save_npz(f'./book_text_features_tfidf/test_{feature.lower()}_vec.npz', vec_test)

create_npz_file()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Publisher
(23063, 3524)
(5766, 3524)


In [17]:
train_df = pd.read_csv(r"book_rating_train.csv", 
                        index_col = False, delimiter = ',', header=0)
train_df['Publisher'].fillna("").apply(lambda x: preprocessing_pipeline(x))

0                     teach resourc
1                         doubleday
2                     chronicl book
3                        bison book
4                  penguin book ltd
                    ...            
23058                         21361
23059         hmh book young reader
23060    rowman littlefield publish
23061            new amsterdam book
23062                        puffin
Name: Publisher, Length: 23063, dtype: object

In [ ]:
import re
seed = 42
def partition():
  df = pd.read_csv(r"book_rating_train.csv", 
                         index_col = False, delimiter = ',', header=0)
  train = pd.DataFrame()
  val = pd.DataFrame()
  for value in pd.unique(df['rating_label']):
    class_df = df[df['rating_label'] == value]
    # print(class_df.shape)
    class_df_val = class_df.sample(int(0.2*len(class_df)), replace = False, random_state = seed)
    # print(class_df_val.shape)
    train_class_df = class_df.drop(class_df_val.index)
    # print(train_class_df.shape)
    train = pd.concat([train, train_class_df])
    val = pd.concat([val, class_df_val])

  train = train.sample(frac = 1, random_state = seed)
  val = val.sample(frac = 1, random_state = seed)


  return train, val


train, val = partition()
print(train.shape, val.shape)


(18452, 10) (4611, 10)


In [ ]:
test_df = pd.read_csv(r"book_rating_test.csv", 
                      index_col = False, delimiter = ',', header=0)
test_df.shape

(5766, 9)

In [ ]:
print(len(train.index))

print(len(val.index))

18452
4611


In [ ]:
import pickle

# create a list of strings

# save the list to a pickle file
with open('train_ind.pkl', 'wb') as f:
    pickle.dump(list(train.index), f)


with open('val_ind.pkl', 'wb') as f:
    pickle.dump(list(val.index), f)

In [ ]:
def partition_npz_file(train, val, test):
  # os.makedirs()
  features = ['Authors', 'Name', 'Description']

  for feature in features:
    print(feature)
    train_target = train[feature].apply(lambda x: preprocessing_pipeline(x))
    val_target = val[feature].apply(lambda x: preprocessing_pipeline(x))
    test_target = test[feature].apply(lambda x: preprocessing_pipeline(x))
    # preprocess text and compute counts
    vectorizer = TfidfVectorizer(stop_words='english').fit(train_target)
        
    # generate counts for a new set of documents
    # vec_train = vectorizer.transform(train_target)
    # vec_val = vectorizer.transform(val_target)
    # print(vec_train.shape)
    # print(vec_val.shape)
    vec_test = vectorizer.transform(test_target)
    print(vec_test.shape)

    if(feature == 'Description'):
      # save_npz(f'./partition/train/train_desc_vec.npz', vec_train)
      # save_npz(f'./partition/val/val_desc_vec.npz', vec_val)
      save_npz(f'./partition/test/test_desc_vec.npz', vec_test)


    else:
      # save_npz(f'./partition/train/train_{feature.lower()}_vec.npz', vec_train)
      # save_npz(f'./partition/val/val_{feature.lower()}_vec.npz', vec_val)
      save_npz(f'./partition/test/test_{feature.lower()}_vec.npz', vec_test)


partition_npz_file(train, val, test_df)

Authors
(5766, 11342)
Name
(5766, 15296)
Description
(5766, 90674)


## doc2vec

doc2vec methods are an extension of word2vec. word2vec maps words to a high-dimensional vector space in such a way that words which appear in similar contexts will be close together in the space. doc2vec does a similar embedding for multi-word passages. The doc2vec (or Paragraph Vector) method was introduced by:

**Le & Mikolov (2014)** Distributed Representations of Sentences and Documents<br>
https://arxiv.org/pdf/1405.4053v2.pdf

The implementation of doc2vec used for this project is from gensim and documented here:<br>
https://radimrehurek.com/gensim/models/doc2vec.html

The size of the output vector is a free parameter. Most implemementations use around 100-300 dimensions, but the best size depends on the problem you're trying to solve with the embeddings and the number of training samples, so you may wish to try different vector sizes. We provided doc2vec features for Name (vec_size = 100), Authors (vec_size = 20) and Description (vec_size = 100). The vectors themselves represent directions in a high-dimensional concept space; the columns do not represent specific words or phrases. Values in the vector are continuous real numbers and can be negative.

In [ ]:
import gensim

# size of the output vector
vec_size = 100

# function to preprocess and tokenize text
def tokenize_corpus(txt, tokens_only=False):
    for i, line in enumerate(txt):
        tokens = gensim.utils.simple_preprocess(line)
        if tokens_only:
            yield tokens
        else:
            yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

# tokenize a training corpus
corpus_name = list(tokenize_corpus(train_corpus_name))

# train doc2vec on the training corpus
model = gensim.models.doc2vec.Doc2Vec(vector_size=vec_size, min_count=2, epochs=40)
model.build_vocab(corpus_name)
model.train(corpus_name, total_examples=model.corpus_count, epochs=model.epochs)

# tokenize new documents
doc = list(tokenize_corpus(test_name, tokens_only=True))

# generate embeddings for the new documents
x_test_name = np.zeros((len(doc),vec_size))
for i in range(len(doc)):
    x_test_name[i,:] = model.infer_vector(doc[i])
    
# check the shape of doc_emb
print(x_test_name.shape)

(23063, 100)


# Text preparation

In [ ]:
import os 
import numpy as np
import pandas as pd

In [ ]:
train_df = pd.read_csv("book_rating_train.csv")
train_df

,Name,Authors,PublishYear,PublishMonth,PublishDay,Publisher,Language,pagesNumber,Description,rating_label
0,Best of Dr Jean: Reading & Writing,Jean R. Feldman,2005,6,1,Teaching Resources,NaN,48,Teachers will turn to this treasury of ideas a...,4.0
1,Here All Dwell Free,Gertrud Mueller Nelson,1991,10,1,DoubleDay,NaN,364,Every human being lives a fairy tale -- an unc...,4.0
2,Boomer's Big Surprise,Constance W. McGeorge,2005,3,31,Chronicle Books,NaN,32,<i>Boomer's Big Surprise</i> will have special...,4.0
3,"I'll Go and Do More: Annie Dodge Wauneka, Nava...",Carolyn Niethammer,2004,9,1,Bison Books,NaN,293,<i>I'll Go and Do More</i> is the story of Ann...,4.0
4,Us,Richard Mason,2005,7,7,Penguin Books Ltd,eng,352,"Since their days at Oxford, they've gone their...",3.0
...,...,...,...,...,...,...,...,...,...,...
23058,Black Coffee Blues,Henry Rollins,1997,8,1,2.13.61,eng,120,"""If I lose the light of the sun, I will write ...",4.0
23059,America's Champion Swimmer: Gertrude Ederle,David A. Adler,2005,6,1,HMH Books for Young Readers,NaN,32,"Trudy Ederle loved to swim, and she was determ...",4.0
23060,Crime and Custom in Savage Society,Bronisław Malinowski,1989,2,15,Rowman & Littlefield Publishers,NaN,132,Bronislaw Malinowski achieved international re...,4.0
23061,The Name and Nature of Poetry and Other Select...,A.E. Housman,1998,4,21,New Amsterdam Books,NaN,136,Lovers of Housman's poetry and admirers of his...,4.0


In [ ]:
import re
def create_directory(df):
  os.makedirs(f"/content/drive/MyDrive/Bookpred/Assignment 2/English")
  for value in pd.unique(df['rating_label']):
    os.makedirs(f"/content/drive/MyDrive/Bookpred/Assignment 2/English/{value}")
    class_df = df[df['rating_label'] == value]

    for i, row in class_df.iterrows():
      description = row['Description']
      clean = re.compile('<.*?>')
      description = re.sub(clean, '', description)

      filename = f"/content/drive/MyDrive/Bookpred/Assignment 2/English/{value}/{np.random.randint(0,1000)}_{np.random.randint(0,1000)}_{np.random.randint(0,100)}.txt"      
      with open(filename, "w") as fp:
        fp.write(description)
    print(class_df.shape)

create_directory(train_df)

(16208, 10)
(5864, 10)
(991, 10)


In [ ]:
len(os.listdir("/content/drive/MyDrive/Bookpred/Assignment 2/English/5.0"))

991

In [ ]:
!find ./ -type f -name "*.txt" -delete

In [ ]:
import os

folder_path = '/content/drive/MyDrive/Bookpred/Assignment 2'  # current folder path
for file in os.listdir(folder_path):
    if file.endswith('.txt'):
        os.remove(os.path.join(folder_path, file))


In [ ]:
import shutil

shutil.rmtree('/content/drive/MyDrive/Bookpred/Assignment 2/English', ignore_errors=True)


In [ ]:
os.listdir()

['book_rating_train.csv',
 'book_rating_test.csv',
 'COMP30027_Project2_spec_2023S1.pdf',
 'random_forest.joblib',
 'random_forest_doc.joblib',
 '.ipynb_checkpoints',
 'book_text_features_doc2vec',
 'book_text_features_countvec',
 'Feature engineering.ipynb',
 'xgb_model.pkl',
 'xgb_model_doc.pkl',
 'random_forest_num.joblib',
 '8.PretrainedBERT_IMDB.ipynb',
 'matrix.npz',
 'book_text_features_tfidf',
 'text_features_description.ipynb',
 'Assignment 2 Track.gdoc',
 'Model.ipynb',
 'output.csv']